In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install click==8.1.7
!pip install cuda-python==11.8.3
!pip install opencv-python==4.9.0.80


  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.8
    Uninstalling click-8.1.8:
      Successfully uninstalled click-8.1.8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: cuda-python
    Found existing installation: cuda-python 12.6.2.post1
    Uninstalling cuda-python-12.6.2.post1:
      Successfully uninstalled cuda-python-12.6.2.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.2.1 requires cuda-python<13.0a0,>=12.6.2, but you have cuda-python 11.8.3 which is incompatible.
pylibcudf-cu12 25.2.1 requires cuda-python<13.0a0,>=12.6.2, but you have cuda-python 11.8.3 which is incompatible.
cuml-cu12 25.2.1 requires cuda-python<13.

In [2]:
# TensorFlow tamamen kaldırılsın (çünkü ihtiyaç yok)
!pip uninstall -y tensorflow tensorflow-estimator keras keras-nightly keras-preprocessing

# NumPy uyumlu sürüme çekilsin (OpenCV, PyTorch vs. ile uyumlu olan)
!pip install numpy==1.24.4

# OpenCV ve tensorboard yeniden yüklensin
!pip install --force-reinstall opencv-python
!pip install --force-reinstall tensorboard==2.14


Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.
cudf-cu12 25.2.1 requires cuda-python<13.0a0,>=12.6.2, but you have cuda-python 11.8.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which 

  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
^C
^C


In [3]:
!git clone https://github.com/cqylunlun/pbas.git


Cloning into 'pbas'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 51 (delta 16), reused 41 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 1.25 MiB | 37.63 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [1]:
!python /content/pbas/main.py --help


INFO:__main__:Command line arguments: /content/pbas/main.py --help
Usage: main.py [OPTIONS] COMMAND1 [ARGS]... [COMMAND2 [ARGS]...]...

Options:
  --results_path TEXT
  --gpu INTEGER        [default: 0]
  --seed INTEGER       [default: 0]
  --log_group TEXT
  --log_project TEXT
  --run_name TEXT
  --test TEXT
  --help               Show this message and exit.

Commands:
  dataset
  net


In [3]:
!python /content/pbas/main.py net --help


INFO:__main__:Command line arguments: /content/pbas/main.py net --help
Usage: main.py net [OPTIONS]

Options:
  --dsc_margin FLOAT
  --train_backbone
  -b, --backbone_names TEXT
  -le, --layers_to_extract_from TEXT
  --pretrain_embed_dimension INTEGER
  --target_embed_dimension INTEGER
  --patchsize INTEGER
  --meta_epochs INTEGER
  --eval_epochs INTEGER
  --dsc_layers INTEGER
  --dsc_hidden INTEGER
  --pre_proj INTEGER
  --k FLOAT
  --lr FLOAT
  --help                          Show this message and exit.


In [4]:
!python /content/pbas/main.py dataset --help


INFO:__main__:Command line arguments: /content/pbas/main.py dataset --help
Usage: main.py dataset [OPTIONS] NAME DATA_PATH

Options:
  -d, --subdatasets TEXT  [required]
  --batch_size INTEGER    [default: 8]
  --num_workers INTEGER   [default: 16]
  --resize INTEGER        [default: 288]
  --imagesize INTEGER     [default: 288]
  --help                  Show this message and exit.


In [2]:
import torch
print("GPU aktif mi?", torch.cuda.is_available())
print("GPU adı:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "YOK")


GPU aktif mi? True
GPU adı: NVIDIA L4


In [4]:
!rm -f /content/pbas/ckpt/mvtec_wood/ckpt.pth


In [3]:
# 1. Dataset yolun
datapath = "/content/drive/MyDrive/wood_otsu"
subdatasets = ["wood"]
flags = ' '.join([f"--subdatasets {cls}" for cls in subdatasets])

# 2. Eğitim komutu
command = f"""
python /content/pbas/main.py \
  --gpu 0 \
  --seed 0 \
  net \
    -b wideresnet50 \
    -le layer2 \
    -le layer3 \
    --pretrain_embed_dimension 1536 \
    --target_embed_dimension 1536 \
    --patchsize 3 \
    --meta_epochs 20 \
    --eval_epochs 5 \
    --dsc_layers 2 \
    --dsc_hidden 1024 \
    --pre_proj 1 \
    --k 0.25 \
  dataset \
    --batch_size 8 \
    --resize 256 \
    --imagesize 256 \
    {flags} mvtec {datapath}
"""

# 3. (İsteğe bağlı) Önceki checkpoint silinsin
!rm -f /content/pbas/ckpt/mvtec_wood/ckpt.pth

# 4. Çalıştır
!{command}


2025-03-29 09:49:34.190806: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-29 09:49:34.209118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743241774.231105   13302 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743241774.237751   13302 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-29 09:49:34.260122: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr